<a href="https://colab.research.google.com/github/SiracencoSerghei/DataScienceHW/blob/main/HW3/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

---
# Write the linear regression hypothesis function in vector form;
напишіть функцію гіпотези лінійної регресії у векторному вигляді;
---

$$h(\overrightarrow{X}) = \overrightarrow{w} \cdot  \overrightarrow{X}$$
or
$$
h(\mathbf{X}) = \mathbf{w}^T \mathbf{X}
$$


In [2]:
def hypothesis(w, X):
  """
    Calculates the predicted values (hypothesis) for linear regression.

    Parameters:
    w (array-like): A 1D array or vector of weights (model coefficients)
                    including the intercept term.
    X (array-like): A 1D or 2D array representing the feature matrix.
                    Each row corresponds to a data point, and each column to a feature.

    Returns:
    float or array-like: The predicted value(s) based on the input features and weights.
                         If X is 1D, a single value is returned.
                         If X is 2D, a 1D array of predictions is returned.

    Explanation:
    - The function computes the dot product of the weights (w) and the features (X),
      which represents the linear combination of the inputs weighted by the model coefficients.
    - This corresponds to the formula h(X) = w_0 + w_1*x_1 + w_2*x_2 + ... + w_n*x_n,
      where w_0 is the intercept, and w_1 to w_n are the feature coefficients.
    """
  return np.dot(w, X)

---
# Create a function to calculate the loss function in vector form;
створіть функцію для обчислення функції втрат у векторному вигляді;
---

$$
Loss = \frac{1}{2n} \| Xw - y \|^2
$$
or
$$
Loss = \frac{1}{2n} \sum_{i=1}^{n} (h(X_i) - y_i)^2
$$



In [ ]:
def loss_func(features, target, weights):
  """
  Calculates the mean squared error (MSE) loss/cost/error for linear regression.

  Parameters:
  target (array-like): A 1D array of target values (dependent variable, y).
  features (array-like): A 2D array where each row represents a data point and each column represents a feature (independent variables, X).
  weights (array-like): A 1D array of model weights (coefficients), including the intercept term.

  Returns:
  float: The computed loss value (MSE) for the given weights and data.

  Explanation:
  - The loss function measures how well the model's predictions match the target values.
  - The formula for the loss is: L(w) = (1 / (2 * n)) * Σ(h(X_i) - y_i)^2
    where:
      - n is the number of data points,
      - h(X_i) = X @ w is the predicted value for the i-th data point,
      - y_i is the actual target value for the i-th data point.
  """
  n = len(target)
  h = hypothesis(features, weights)
  loss = (1/(2*n)) * np.sum((h - target)**2)
  return loss

---
# Implement one step of gradient descent;
реалізуйте один крок градієнтного спуску;
---

In [4]:
def gradient_descent_step(X, y, w, learning_rate=.001):
    """
    Performs a single step of gradient descent to update the model's weights.

    Parameters:
    w (array-like): The current weights of the model.
    X (array-like): The feature matrix.
    y (array-like): The target values.
    learning_rate (float): The learning rate (step size) for the gradient descent.

    Returns:
    array-like: The updated weights after one gradient descent step.
    """
    n = len(y)
    # Обчислюємо похибку
    h = hypothesis(w, X)
    error = h - y
    # Calculate the gradient of the loss function
    gradient = (1 / n) * np.dot(X.T, error)

    # Update the weights using the gradient and learning rate
    w -= learning_rate * gradient
    return w

In [6]:
def gradient_descent(X, y, w, num_iterations=1, learning_rate=0.001):
    """
    Implements gradient descent for linear regression.

    Parameters:
    X (array-like): Feature matrix (independent variables), dimensions (m, n).
    y (array-like): Target vector (dependent variable), dimensions (m, 1).
    w (array-like): Weight vector (coefficients), dimensions (n, 1).
    num_iterations (int): Number of iterations to update the weights.
    learning_rate (float): Step size for gradient descent.

    Returns:
    tuple:
        - w: The final weight vector after optimization.
        - loss_history: An array storing the loss at each iteration.
    """
    # Initialize an array to store the loss at each iteration
    loss_history = np.zeros(num_iterations)

    for i in range(num_iterations):
        # Perform a single gradient descent step to update weights
        w = gradient_descent_step(X, y, w, learning_rate)

        # Compute the loss (Mean Squared Error) for the current weights
        loss_history[i] = loss_func(X, y, w)

    return w, loss_history

